## Final Report

In [1]:
import pandas as pd
import numpy as np
import wrangle as w

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)

In [2]:
#get the data
df = w.acquire_anonymized_curriculum_access_data()

In [3]:
#look at the data
df

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,2014-02-04,2014-02-04,2.0


In [4]:
#look at the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   page        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   name        847330 non-null  object 
 7   start_date  847330 non-null  object 
 8   end_date    847330 non-null  object 
 9   program_id  847330 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 68.7+ MB


In [5]:
# clean the data
df = w.clean_the_data(df)

## 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

**Findings:**
- Data Science accessed classification/overview 1785 times.
- Front End accessed content/html-css 2 times.
- Java accessed javascript-i 17457 times.
- PHP accessed javascript-i 736 times.

In [6]:
#Make df for each program
ds=df[df.program_id=='Data Science']
fe=df[df.program_id=='Front End']
jv=df[df.program_id=='Java']
php=df[df.program_id=='PHP']

In [7]:
#Look at what page was most viewed for data science
ds.groupby(['page'])['page'].count().sort_values(ascending=False)

page
/                                                                                                                                                                                                                                                                                                                         8358
search/search_index.json                                                                                                                                                                                                                                                                                                  2203
classification/overview                                                                                                                                                                                                                                                                                                   1785
1-fundamentals/modern-data-scientist.j

In [8]:
#Look at what page was most viewed for front end
fe.groupby(['page'])['page'].count().sort_values(ascending=False)

page
content/html-css                               2
/                                              1
content/html-css/gitbook/images/favicon.ico    1
content/html-css/introduction.html             1
Name: page, dtype: int64

In [9]:
#Look at what page was most viewed for java
jv.groupby(['page'])['page'].count().sort_values(ascending=False)

page
/                                                                      35814
javascript-i                                                           17457
toc                                                                    17428
search/search_index.json                                               15212
java-iii                                                               12683
                                                                       ...  
content/mysql/relationships/sub-queries.html                               1
content/mysql/sample-database/gitbook/images/favicon.ico                   1
content/php_iii/classes-and-objects-i/constructors-destructors.html        1
content/php_iv/classes-and-objects-ii/late-static-binding.html             1
wp-login                                                                   1
Name: page, Length: 1913, dtype: int64

In [10]:
#Look at what page was most viewed for php
php.groupby(['page'])['page'].count().sort_values(ascending=False)

page
/                                                                                1681
index.html                                                                       1011
javascript-i                                                                      736
html-css                                                                          542
spring                                                                            501
java-iii                                                                          479
java-ii                                                                           454
java-i                                                                            444
javascript-ii                                                                     429
appendix                                                                          409
jquery                                                                            344
mysql                                            

## 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

**Findings:**
- Cohorts **Kings, Quincy, and Joshua** all look at PHP in their top 10 most visited lessons. They also have differently marked pathways to their content which could suggest that they made significant changes before or after these cohorts.
- Regression lesson is most viewed for **Curie** with about 600 clicks where as it barely shows up one other time in the top 6 lessons for other data science cohorts

In [11]:
#make df of just those in Data Science 
DS = df[df.program_id == 'Data Science']

#make df of just those in WebDev 
WebDev = df[(df.program_id == 'PHP') | (df.program_id == 'Java') | (df.program_id == 'Front End') ]

In [12]:
DS_Cohorts = DS.groupby('page')['name'].count().sort_values()

In [13]:
DS_Cohorts

page
%20https://github.com/RaulCPena                                                                                                                                                                                                                                                                                              1
appendix/open_data/www.census.gov                                                                                                                                                                                                                                                                                            1
appendix/open_data/www.flickr.com/services/api                                                                                                                                                                                                                                                                               1
appendix/open_data/www.openstreetmap.o

In [14]:
WebDev_Cohorts = WebDev.groupby('page')['name'].count().sort_values()

In [15]:
WebDev_Cohorts

page
'                               1
cli-02-listing-files            1
cli-03-file-paths               1
cli-07-more-topics              1
codeup-exercises                1
                            ...  
java-iii                    13162
search/search_index.json    15331
toc                         17580
javascript-i                18193
/                           37496
Name: name, Length: 2054, dtype: int64

In [16]:
WebDevDict = {}
for x in WebDev.name.unique():
    WebDevDict[x] = pd.DataFrame(df[df.name == x])

In [17]:
DSDict = {}
for x in DS.name.unique():
    DSDict[x] = pd.DataFrame(df[df.name == x])

In [18]:
#cohorts Kings, quincy, and joshua all look at PHP in their top 10 most visited lessons. they also have differently marked pathways to their content which could suggest that they made significant changes before or after these cohorts.
for x in WebDevDict:
    display(x, WebDevDict[x].groupby('page')['name'].count().sort_values(ascending=False).head(10))


'Hampton'

page
/                    210
java-iii              57
appendix              55
java-i                46
java-ii               46
spring                43
javascript-i          36
javascript-ii         31
java-iii/servlets     30
mysql                 30
Name: name, dtype: int64

'Teddy'

page
/                           1828
spring                       744
java-iii                     732
mysql                        638
javascript-i                 615
mkdocs/search_index.json     595
java-i                       522
jquery                       512
java-ii                      509
appendix                     505
Name: name, dtype: int64

'Sequoia'

...

page
toc                                                                          910
javascript-i                                                                 907
/                                                                            699
javascript-i/introduction/working-with-data-types-operators-and-variables    505
html-css                                                                     479
javascript-i/javascript-with-html                                            411
jquery                                                                       405
javascript-i/functions                                                       382
javascript-i/loops                                                           312
javascript-i/conditionals                                                    292
Name: name, dtype: int64

'Neptune'

page
/                                                                            472
toc                                                                          451
javascript-i/introduction/working-with-data-types-operators-and-variables    329
html-css                                                                     313
javascript-i/javascript-with-html                                            311
javascript-i                                                                 272
html-css/css-ii/bootstrap-grid-system                                        256
html-css/css-ii/bootstrap-introduction                                       227
javascript-i/loops                                                           226
javascript-i/functions                                                       217
Name: name, dtype: int64

'Oberon'

page
javascript-i/introduction/operators                                          179
javascript-i/introduction/primitive-types                                    177
javascript-i/functions                                                       171
javascript-i/introduction/working-with-data-types-operators-and-variables    146
javascript-i/conditionals                                                    144
javascript-i                                                                 136
/                                                                            131
javascript-i/javascript-with-html                                            125
toc                                                                          101
javascript-i/introduction/variables                                           95
Name: name, dtype: int64

In [19]:
# Regression lesson in most viewed for Curie  with ab 600 clicks where as it barely shows up one other time in the top 6 lessons for other data science cohorts
for x in DSDict:
    display(x, DSDict[x].groupby('page')['name'].count().sort_values(ascending=False).head(7))


'Bayes'

page
/                                                    1967
1-fundamentals/modern-data-scientist.jpg              650
1-fundamentals/AI-ML-DL-timeline.jpg                  648
1-fundamentals/1.1-intro-to-data-science              640
search/search_index.json                              588
6-regression/1-overview                               521
10-anomaly-detection/AnomalyDetectionCartoon.jpeg     387
Name: name, dtype: int64

'Curie'

page
/                                           1712
6-regression/1-overview                      595
search/search_index.json                     538
1-fundamentals/modern-data-scientist.jpg     467
1-fundamentals/AI-ML-DL-timeline.jpg         465
1-fundamentals/1.1-intro-to-data-science     461
3-sql/1-mysql-overview                       441
Name: name, dtype: int64

'Darden'

page
/                                                 2980
classification/overview                           1109
classification/scale_features_or_not.svg           943
sql/mysql-overview                                 774
search/search_index.json                           664
anomaly-detection/AnomalyDetectionCartoon.jpeg     612
anomaly-detection/overview                         592
Name: name, dtype: int64

'Easley'

page
/                                                                1115
classification/scale_features_or_not.svg                          463
classification/overview                                           445
classification/classical_programming_vs_machine_learning.jpeg     432
fundamentals/AI-ML-DL-timeline.jpg                                381
fundamentals/modern-data-scientist.jpg                            379
fundamentals/intro-to-data-science                                372
Name: name, dtype: int64

'Florence'

page
fundamentals/modern-data-scientist.jpg    627
fundamentals/AI-ML-DL-timeline.jpg        624
fundamentals/intro-to-data-science        615
/                                         584
python/data-types-and-variables           258
fundamentals/DataToAction_v2.jpg          208
sql/mysql-overview                        203
Name: name, dtype: int64

## 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

**Findings:**
- There are 233 users that didn't access the curriculum during their cohort timeframe, but some of those don't have a cohort listed (78)
- I defined "hardly active" as those students who accessed less than or equal to 10% of the average usage (pages viewed)
- 54 users accessed less than or equal to 10% of the average number of pages viewed per user.
- All but one user was from the Java program, user_id 679 was data science(assumed quit the course)
- **Oberon** and **Sequoia** cohorts only had students access the curriculum up through the first month of the course. ***This might mean that these cohorts used a different curriculum platform or there is something wrong with the start and end dates in the db***
- user_ids 24, 41, 371, 741, 56, 57, 45, 36, 250, 65, 52, 33, 253, 278 only started accessing the curriculum when the cohort was almost finished 
- user_ids 879, 832,  388, 572, 772, 529, 543, 46, 13, 702, 806 accessed the curriculum on the first day (or first week) and then stopped. ***I assume these are students who dropped out.***



Below is the code that helped me discover these finidings.

In [20]:
#only look at students
students = df[df['name'] != 'Staff']

In [21]:
#find those who accessed the curriculum during their class
active_student_access = students.query('start_date < date < end_date')

In [22]:
usage = active_student_access.user_id.value_counts().sort_values()

In [23]:
#I defined "hardly active" as those students who accessed less than or equal to 10% of the average usage (pages viewed)
usage.mean() * .10

89.00359116022099

In [24]:
#how many accessed less than 10% of the average -- 54
usage[usage <= 89]

879     1
679     3
832     3
278     4
956     5
812     7
388     8
572    12
889    16
968    16
487    17
963    17
961    18
772    20
49     25
64     25
976    25
24     26
41     28
972    31
973    32
967    33
529    35
959    38
971    41
964    42
371    42
981    42
960    45
741    47
978    49
965    51
56     52
962    52
543    53
46     54
57     62
969    62
858    63
45     65
36     65
975    66
250    67
65     73
52     75
977    76
974    76
13     79
33     79
955    80
702    85
806    88
954    88
253    89
Name: user_id, dtype: int64

In [25]:
#test oddities of those who didn't view many times during cohort
active_student_access[active_student_access.user_id == 24]

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-01-26,2018-01-26,11:24:17,appendix,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-26,2018-01-26,11:24:29,appendix/capstone-workbook,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,11:25:49,/,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,11:25:54,spring,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,11:25:59,spring/fundamentals/views,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:28:23,mkdocs/search_index.json,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:28:33,html-css,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:28:41,html-css/css-ii/bootstrap-introduction,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java
2018-01-29,2018-01-29,13:29:50,/,24,21.0,97.105.19.61,Sequoia,2017-09-27,2018-02-15,Java


## 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

## 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

## 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

<div class="alert alert-info">

    
# Q#6 
# ANSWER:

## 1. Data Science
    
|Curriculum        |Click Count|Student Count|
|:-----------------|:-:|:-:|
|sql/mysql-overview|275|38|
|classification/overview|266|39|
|anomaly-detection/overview|191|31|
|fundamentals/intro-to-data-science|184|36|

## 2. PHP

|Curriculum        |Click Count|Student Count|
|:-----------------|:-:|:-:|
|javascript-i	|736|	35|
|html-css	|542|	40|
|spring	|501|	25|
|java-iii	|479|	20|
    
## 3. Java
    
|Curriculum        |Click Count|Student Count|
|:-----------------|:-:|:-:|    
|javascript-i	|4229|	300|
|spring	|3760|	306|
|html-css	|3136|	296|
|java-iii	|3058|	273|
    
## 4. Front End    
    
|Curriculum        |Click Count|Student Count|
|:-----------------|:-:|:-:|    
|content/html-css	|2	|1|
|content/html-css/introduction.html	|1	|1|    

# Grad Users

In [26]:
# Create DataFrame for users that have accessed the curriculum past there graduation date
grad_users = df[df.date > df.end_date]

In [27]:
grad_users.T

date,2018-01-26,2018-01-26,2018-01-26,2018-01-26,2018-01-26,2018-01-26,2018-01-26,2018-01-26,2018-01-26,2018-01-26,...,2021-04-21,2021-04-21,2021-04-21,2021-04-21,2021-04-21,2021-04-21,2021-04-21,2021-04-21,2021-04-21,2021-04-21
date,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,2018-01-26 00:00:00,...,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00,2021-04-21 00:00:00
time,09:55:03,09:56:02,09:56:05,09:56:06,10:14:47,10:14:51,10:15:02,10:40:15,10:59:02,10:59:06,...,16:07:17,16:30:30,16:41:29,16:41:31,16:41:49,16:41:51,16:42:02,16:42:09,16:44:37,16:44:39
page,/,java-ii,java-ii/object-oriented-programming,slides/object_oriented_programming,/,javascript-i,javascript-i/functions,javascript-i/functions,index.html,java-i,...,javascript-ii/RESTful-api,appendix,javascript-i,javascript-ii,jquery,jquery/personal-site,jquery/mapbox-api,jquery/ajax/weather-map,anomaly-detection/discrete-probabilistic-methods,jquery/mapbox-api
user_id,1,1,1,1,11,11,11,1,21,21,...,64,771,64,64,64,64,64,64,744,64
cohort_id,8.0,8.0,8.0,8.0,1.0,1.0,1.0,8.0,16.0,16.0,...,28.0,62.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0
ip,97.105.19.61,97.105.19.61,97.105.19.61,97.105.19.61,97.105.19.61,97.105.19.61,97.105.19.61,97.105.19.61,71.137.146.71,71.137.146.71,...,71.150.217.33,172.125.226.175,71.150.217.33,71.150.217.33,71.150.217.33,71.150.217.33,71.150.217.33,71.150.217.33,24.160.137.86,71.150.217.33
name,Hampton,Hampton,Hampton,Hampton,Arches,Arches,Arches,Hampton,Niagara,Niagara,...,Staff,Jupiter,Staff,Staff,Staff,Staff,Staff,Staff,Staff,Staff
start_date,2015-09-22 00:00:00,2015-09-22 00:00:00,2015-09-22 00:00:00,2015-09-22 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2015-09-22 00:00:00,2016-10-26 00:00:00,2016-10-26 00:00:00,...,2014-02-04 00:00:00,2020-09-21 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00
end_date,2016-02-06 00:00:00,2016-02-06 00:00:00,2016-02-06 00:00:00,2016-02-06 00:00:00,2014-04-22 00:00:00,2014-04-22 00:00:00,2014-04-22 00:00:00,2016-02-06 00:00:00,2017-03-09 00:00:00,2017-03-09 00:00:00,...,2014-02-04 00:00:00,2021-03-30 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00,2014-02-04 00:00:00
program_id,PHP,PHP,PHP,PHP,PHP,PHP,PHP,PHP,Java,Java,...,Java,Java,Java,Java,Java,Java,Java,Java,Java,Java


In [28]:
# LOOK AT GRAD USERS
grad_program_views = grad_users.groupby(['program_id'])['page'].agg(['count', 'nunique'])
grad_program_views.sort_values(by='count', ascending=False)

,count,nunique
program_id,,
Java,157786,1656
PHP,30548,710
Data Science,11495,437
Front End,5,4


In [29]:
# LOOK AT WHOLE DF
df_program_views = df.groupby(['program_id'])['page'].agg(['count', 'nunique'])
df_program_views.sort_values(by='count', ascending=False)

,count,nunique
program_id,,
Java,713365,1913
Data Science,103411,682
nan,52893,1112
PHP,30548,710
Front End,5,4


In [30]:
grad_users.groupby('program_id')['page'].value_counts()

program_id    page                                    
Data Science  /                                           1436
              search/search_index.json                     493
              sql/mysql-overview                           275
              classification/overview                      266
              classification/scale_features_or_not.svg     219
                                                          ... 
PHP           students                                       1
              students/468/notes                             1
              students/units/75/sub_units/268                1
              teams/13                                       1
              uploads/58a217a705bde.jpg                      1
Name: page, Length: 2807, dtype: int64

# Data Science

In [31]:
# Create DataFrame for Data Science users that have accessed the curriculum past there graduation date
ds_grad_users = grad_users[grad_users.program_id == 'Data Science']

In [32]:
# Count how many times each page was accessed
ds_grad_users.page.value_counts().head(10)

/                                                 1436
search/search_index.json                           493
sql/mysql-overview                                 275
classification/overview                            266
classification/scale_features_or_not.svg           219
anomaly-detection/AnomalyDetectionCartoon.jpeg     193
anomaly-detection/overview                         191
fundamentals/AI-ML-DL-timeline.jpg                 189
fundamentals/modern-data-scientist.jpg             187
fundamentals/intro-to-data-science                 184
Name: page, dtype: int64

In [33]:
# Look at curriculum by click count and student nunique
ds_program_views = ds_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
ds_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
/,1436,57
search/search_index.json,493,42
sql/mysql-overview,275,38
classification/overview,266,39
classification/scale_features_or_not.svg,219,36
anomaly-detection/AnomalyDetectionCartoon.jpeg,193,31
anomaly-detection/overview,191,31
fundamentals/AI-ML-DL-timeline.jpg,189,36
fundamentals/modern-data-scientist.jpg,187,35


## Check the min/max Graduation Dates.

In [34]:
# Create Java for DS users
DS_users = df[df.program_id == 'Data Science']

In [35]:
# show max end date for DS users
DS_users.end_date.min()

Timestamp('2020-01-30 00:00:00')

In [36]:
# show max end date for DS users
DS_users.end_date.max()

Timestamp('2021-09-03 00:00:00')

In [37]:
# show min access date for df
df.date.min()

Timestamp('2018-01-26 00:00:00')

# PHP

In [38]:
# Create DataFrame for PHP users that have accessed the curriculum past there graduation date
php_grad_users = grad_users[grad_users.program_id == 'PHP']

In [39]:
# Look at curriculum by click count and student nunique
php_program_views = php_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
php_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
/,1681,90
index.html,1011,20
javascript-i,736,35
html-css,542,40
spring,501,25
java-iii,479,20
java-ii,454,24
java-i,444,40
javascript-ii,429,33


## Why are Curriculum access counts the same for PHP during and after graduation?

In [40]:
# Create DataFrame for PHP users
php_users = df[df.program_id == 'PHP']

In [41]:
# show max end date for Front End users
php_users.end_date.max()

Timestamp('2017-09-22 00:00:00')

In [42]:
# show min access date for df
df.date.min()

Timestamp('2018-01-26 00:00:00')

In [43]:
# Count how many times each page was accessed
php_users.page.value_counts().head(10)

/                1681
index.html       1011
javascript-i      736
html-css          542
spring            501
java-iii          479
java-ii           454
java-i            444
javascript-ii     429
appendix          409
Name: page, dtype: int64

## ANSWER: Because the last of the PHP users graduated in 2017, before the start date of our data access collection which begins in 2018.

# JAVA

In [44]:
# Create DataFrame for Java users that have accessed the curriculum past there graduation date
java_grad_users = grad_users[grad_users.program_id == 'Java']

In [45]:
# Count how many times each page was accessed
java_grad_users.page.value_counts().head(10)

/                           12406
javascript-i                 4229
spring                       3760
search/search_index.json     3562
html-css                     3136
java-iii                     3058
java-ii                      2985
java-i                       2679
appendix                     2662
javascript-ii                2549
Name: page, dtype: int64

In [46]:
# Look at curriculum by click count and student nunique
java_program_views = java_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
java_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
/,12406,406
javascript-i,4229,300
spring,3760,306
search/search_index.json,3562,262
html-css,3136,296
java-iii,3058,273
java-ii,2985,274
java-i,2679,300
appendix,2662,279


## Check the min/max Graduation Dates.

In [47]:
# Create Java for PHP users
java_users = df[df.program_id == 'Java']

In [48]:
# show max end date for Java users
java_users.end_date.min()

Timestamp('2014-02-04 00:00:00')

In [49]:
# show max end date for Java users
java_users.end_date.max()

Timestamp('2021-10-01 00:00:00')

In [50]:
# show min access date for df
df.date.min()

Timestamp('2018-01-26 00:00:00')

# Front End

In [51]:
# Create DataFrame for Front End users that have accessed the curriculum past there graduation date
fe_grad_users = grad_users[grad_users.program_id == 'Front End']

In [52]:
# Count how many times each page was accessed
fe_grad_users.page.value_counts().head(10)

content/html-css                               2
/                                              1
content/html-css/gitbook/images/favicon.ico    1
content/html-css/introduction.html             1
Name: page, dtype: int64

In [53]:
# Look at curriculum by click count and student nunique
fe_program_views = fe_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
fe_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
content/html-css,2,1
/,1,1
content/html-css/gitbook/images/favicon.ico,1,1
content/html-css/introduction.html,1,1


In [54]:
# Look at curriculum by click count and student nunique
ds_program_views = ds_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
ds_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
/,1436,57
search/search_index.json,493,42
sql/mysql-overview,275,38
classification/overview,266,39
classification/scale_features_or_not.svg,219,36
anomaly-detection/AnomalyDetectionCartoon.jpeg,193,31
anomaly-detection/overview,191,31
fundamentals/AI-ML-DL-timeline.jpg,189,36
fundamentals/modern-data-scientist.jpg,187,35


In [55]:
# Look at curriculum by click count and student nunique
php_program_views = php_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
php_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
/,1681,90
index.html,1011,20
javascript-i,736,35
html-css,542,40
spring,501,25
java-iii,479,20
java-ii,454,24
java-i,444,40
javascript-ii,429,33


In [56]:
# Look at curriculum by click count and student nunique
java_program_views = java_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
java_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
/,12406,406
javascript-i,4229,300
spring,3760,306
search/search_index.json,3562,262
html-css,3136,296
java-iii,3058,273
java-ii,2985,274
java-i,2679,300
appendix,2662,279


In [57]:
# Look at curriculum by click count and student nunique
fe_program_views = fe_grad_users.groupby(['page'])['user_id'].agg(['count', 'nunique'])
fe_program_views.sort_values(by='count', ascending=False).head(10)

,count,nunique
page,,
content/html-css,2,1
/,1,1
content/html-css/gitbook/images/favicon.ico,1,1
content/html-css/introduction.html,1,1


## 7. Which lessons are least accessed?

**Findings:**
- Data Science accessed pages with the prefix appendix the least.
- Front End accessed pages with the prefix content/html-css the least.
- Java accessed pages with the prefix cli- the least.
- PHP accessed pages with the prefix content/appendix/javascript/functions/ the least.

In [58]:
#Look at what page was least viewed for data science
ds.groupby(['page'])['page'].count().sort_values()

page
%20https://github.com/RaulCPena                                                                                                                                                                                                                                                                                              1
appendix/open_data/www.census.gov                                                                                                                                                                                                                                                                                            1
appendix/open_data/www.flickr.com/services/api                                                                                                                                                                                                                                                                               1
appendix/open_data/www.openstreetmap.o

In [59]:
#Look at what page was least viewed for front end
fe.groupby(['page'])['page'].count().sort_values()

page
/                                              1
content/html-css/gitbook/images/favicon.ico    1
content/html-css/introduction.html             1
content/html-css                               2
Name: page, dtype: int64

In [60]:
#Look at what page was least viewed for java
jv.groupby(['page'])['page'].count().sort_values()

page
'                                            1
cheatsheet.tex                               1
classification/user-defined-functions        1
cli-02-listing-files                         1
cli-03-file-paths                            1
                                         ...  
java-iii                                 12683
search/search_index.json                 15212
toc                                      17428
javascript-i                             17457
/                                        35814
Name: page, Length: 1913, dtype: int64

In [61]:
#Look at what page was least viewed for php
php.groupby(['page'])['page'].count().sort_values()

page
html-css/media-queries                                                              1
prework/cli/07-editing-files-with-vim                                               1
content/appendix/javascript/functions/controllers.html                              1
content/appendix/javascript/functions/models.html                                   1
content/appendix/javascript/functions/scope.html                                    1
content/appendix/javascript/functions/templating.html                               1
content/appendix/javascript/javascript/functions/scope.html                         1
content/mysql/relationships/indexes.html                                            1
content/appendix/code-standards/html-css.html                                       1
content/appendix/postwork/mars-rover.html                                           1
content/appendix/postwork/trains.html                                               1
content/conditionals.html                        

## 8. Anything else I should be aware of?

### Why are there so few Front End Graduates accessing the Curriculum?

In [62]:
# Create DataFrame for Front End users
fe_users = df[df.program_id == 'Front End']

In [63]:
# show max end date for Front End users
fe_users.end_date.max()

Timestamp('2015-07-29 00:00:00')

In [64]:
# show min access date for df
df.date.min()

Timestamp('2018-01-26 00:00:00')

In [65]:
# How many users and cohorts make up the Front End
fe_users.groupby(['name'])['user_id'].agg(['nunique'])

,nunique
name,
Apollo,1


<div class="alert alert-info">


## ANSWER: Because there was only one Front End user who graduated in 2015, three years before the start date of our data access collection which begins in 2018.

## Most likely, the majority of the Null entries for Cohort could probably be attributed to Front End as well as a few towards Staff